# 환경설정

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [3]:
import matplotlib.pyplot as plt

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [5]:
kospi_index = pd.read_excel('https://github.com/SNMHZ/DART_8TH/raw/master/mock_investing/data/kospi_index.xlsx', index_col=0)

In [6]:
kospi_list = pd.read_excel('https://github.com/SNMHZ/DART_8TH/raw/master/mock_investing/data/kospi_list.xlsx', index_col=0)

In [7]:
%%time
pricedata = pd.read_excel('https://github.com/SNMHZ/DART_8TH/raw/master/mock_investing/data/revised_price.xlsx', index_col=0)

CPU times: user 24.5 s, sys: 221 ms, total: 24.7 s
Wall time: 26.3 s


In [8]:
ticker_name = pd.read_excel('https://github.com/SNMHZ/DART_8TH/raw/master/mock_investing/data/ticker_name.xlsx', index_col=0)

In [9]:
pricedata_kospi = pricedata[kospi_list.columns]

# 함수

In [10]:
#########################   베타 획득   #########################
#######input
#ticker -> 목표 종목 코드(string)
#priceData -> 가격 정보 데이터프레임. 시계열
#market -> 시장 지수 데이터프레임. 시계열
#period -> 관측 목표 기간
#end -> 관측 마지막 시점
#show_plot -> 베타 그래프 출력 유무. True시 출력.

#######output
#베타(float)
#################################################################
def getBeta(ticker, priceData, market, period, end=0, show_plot=False):
  price_change = priceData[ticker].pct_change()[1:]
  market_change = market[market.columns[0]].pct_change()[1:]
  
  start = end - period

  price_change_sliced = price_change.iloc[start:end]
  market_change_sliced = market_change.iloc[start:end]

  if show_plot:
    model = LinearRegression(normalize=True)
    model.fit(pd.DataFrame(market_change_sliced),pd.DataFrame(price_change_sliced))

    plt.figure(figsize=(7, 7))
    plt.grid(True)
    plt.scatter(market_change_sliced, price_change_sliced, color='black', alpha=0.5)
    aran = np.arange(min(market_change_sliced) ,max(market_change_sliced) , 0.001)
    plt.plot(aran, model.predict(aran.reshape(-1, 1)), color='blue')
    
    plt.show()

  return (np.cov(price_change_sliced, market_change_sliced)/np.var(market_change_sliced))[0,1]

In [11]:
######################   Bag of Beta 획득   #####################
#######input
#ticker -> 목표 종목 코드(string)
#observation_time -> 관측을 하는 시점(int)
#observation_period -> 관측할 기간(int)
#observation_step -> 관측 step(int)

#######output
#state -> '종목코드_관측시점'. ex) 'A005930_2016-12-08'(string)
#BoB -> 기간 베타 리스트. [230베타, 225베타 220베타, ... , 10~15베타, 5~10베타, 0~5베타](list)
#################################################################
def getBoB(ticker, observation_time, observation_period=230, observation_step=5):
  try:
    BoB = []
    current_period = observation_time

    #관측기간과 스텝에 따른 관측할 베타 수
    count_step = observation_period//observation_step
    #230, 225, 220 ... 5. 기간별 베타.
    BoB_period = [x*5 for x in reversed(range(1, count_step+1))]
    #460, 455, 450 ... 235. 스텝 베타.
    BoB_time = [x*5+observation_period for x in reversed(range(1, count_step))]

    #기간 베타. 230, 225, 220 ... 5일간 관측 베타
    for period in BoB_period:
      BoB.append( getBeta(ticker, pricedata_kospi, kospi_index, period, observation_time) )

    #스텝 베타 벡터. 230~235, 235~240, 240~245 ... 베타 벡터
    for period in BoB_time:
      BoB.append( getBeta(ticker, pricedata_kospi, kospi_index, observation_step, period) )

    state = ticker+'_'+str(pricedata_kospi.index[observation_time])[:10]
    return state, BoB
  except:
    pass

In [12]:
#Period Yield
def getPY(start, end):
  return (end - start)/start

In [13]:
########################   result 획득   ########################
#######input
#ticker -> 목표 종목 코드(string)
#observation_time -> 관측을 하는 시점(int)
#forward_period -> 수익률 계산 기간(int)

#######output
#앞으로 forward_period 기간이 지난 후
#(개별 기간 수익률 상승) and (벤치마크 수익률 상회)
#True, False (bool)
#################################################################
def getClassified(ticker, observation_time, forward_period=50):
  price_start = pricedata_kospi[ticker].iloc[observation_period]
  price_end = pricedata_kospi[ticker].iloc[observation_period+forward_period]
  market_start = kospi_index['코스피'].iloc[observation_period]
  market_end = kospi_index['코스피'].iloc[observation_period+forward_period]
  return getPY(price_start, price_end) > 0 and getPY(price_start, price_end) > getPY(market_start, market_end)

# 랜덤포레스트 활용


In [14]:
observation_period = 230
observation_step = 5
forward_period = 50

## 230시점(2016-12-08) 학습 후 정확도 확인

In [15]:
period_list = [x*230 for x in range(1, 2)]
print(period_list)

[230]


### 데이터셋 생성

In [16]:
%%time
#230시점 생성
index_list = []
BoB_list = []
for period in period_list:
  print(period)
  for i, ticker in enumerate(pricedata_kospi.columns):
    print('\r%d/%d'%(i+1, len(pricedata_kospi.columns)), end='')
    state, getedBoB = getBoB(ticker, period)
    index_list.append(state)
    BoB_list.append(getedBoB)
  print()

230
784/784
CPU times: user 2min 5s, sys: 1.98 s, total: 2min 7s
Wall time: 2min 4s


In [17]:
BagOfBeta_df_230 = pd.DataFrame(BoB_list, index = index_list)

In [18]:
classified_list = []
for period in period_list:
  print(period)
  for i, ticker in enumerate(pricedata_kospi.columns):
    print('\r%d/%d'%(i+1, len(pricedata_kospi.columns)), end='')
    classified_list.append(getClassified(ticker, period))
  print()

230
784/784


In [19]:
BagOfBeta_df_230['result'] = classified_list
BagOfBeta_df_230_droped = BagOfBeta_df_230.dropna() 

In [20]:
BagOfBeta_df_230_droped 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,result
A005930_2016-12-08,1.338226,1.338483,1.362913,1.357541,1.347650,1.390923,1.460837,1.462274,1.473762,1.451128,1.455293,1.461940,1.457865,1.424735,1.443555,1.462362,1.453106,1.490183,1.485212,1.488197,1.513174,1.532893,1.555864,1.600899,1.783054,1.767923,1.805102,1.804722,1.793388,1.796138,1.806284,1.764772,1.748480,1.754870,1.714398,1.427788,1.482753,1.451303,1.277072,1.300163,...,3.026271,0.749069,1.785682,0.927805,0.648878,2.555923,1.910395,2.558089,2.566363,2.253052,0.575459,1.648875,2.940480,3.602700,1.894304,3.192075,3.276317,2.110682,1.296920,0.638460,6.622136,1.504686,-0.070883,0.602847,1.239400,2.000374,1.681891,1.404396,3.992485,0.317403,1.186953,-0.770792,1.987424,4.161994,2.901080,3.051549,1.056148,4.917341,1.980648,True
A000660_2016-12-08,1.254750,1.242087,1.220905,1.143546,1.168918,1.143197,1.176132,1.193249,1.155881,1.148405,1.159449,1.140309,1.151419,1.177920,1.217678,1.230552,1.181741,1.136567,1.147498,1.154808,1.140827,1.130129,1.133423,1.093474,1.279581,1.252857,1.232169,1.243898,1.294308,1.291827,1.270836,1.254326,1.320058,1.321684,1.359132,1.158966,1.110178,1.219141,1.288386,1.343966,...,1.191958,0.079624,0.004111,2.813305,0.870361,3.978080,2.311774,3.508367,6.727803,4.494073,5.866851,4.207799,5.680777,1.967961,2.778888,1.444210,0.556872,2.596057,2.345116,0.681337,-3.748944,0.844256,2.630603,7.742632,-3.423024,-3.001194,3.330272,1.446278,8.090961,-1.354777,-6.291540,3.270467,1.801854,3.203848,1.669019,2.933238,0.257831,6.100980,1.263442,True
A035420_2016-12-08,0.861434,0.880686,0.942875,0.929793,1.022238,0.944639,0.941297,0.952954,0.872667,0.851025,0.862823,0.849210,0.825198,0.819878,0.834658,0.813038,0.837336,0.859475,0.865122,0.881773,0.882787,0.888311,0.861548,0.862927,0.990403,1.048864,1.090183,1.123572,1.138587,1.200190,1.136596,1.130692,1.124013,1.137770,1.104624,1.220170,1.203678,1.273073,1.140140,1.147087,...,2.878658,4.180823,0.768748,2.331106,-3.247676,-0.472207,2.249312,1.469208,0.507057,2.271955,1.748778,0.134949,1.943806,-3.400550,7.860462,2.068875,0.296454,3.092196,4.375335,1.639354,1.572983,3.632818,1.944998,8.086779,-3.652865,4.886665,2.332576,2.018358,0.104550,0.839243,3.246921,0.888813,-1.509459,1.284209,1.016063,-1.135750,2.174155,5.399600,-0.778535,False
A051910_2016-12-08,1.481199,1.455214,1.423273,1.493332,1.411062,1.378351,1.364039,1.351593,1.331118,1.348830,1.334074,1.355190,1.323500,1.293191,1.255131,1.274977,1.283548,1.271304,1.228539,1.232156,1.246313,1.239002,1.270530,1.253393,1.067992,0.989141,0.942065,0.932957,0.907033,0.827452,0.836157,0.856197,0.907921,0.934806,1.012658,1.163759,1.231378,1.381751,1.452017,1.403880,...,-0.853174,-0.804991,0.018259,3.268729,5.778220,-0.196287,2.634002,2.013340,4.998720,6.990801,4.079725,1.925127,2.555923,0.294914,1.933539,-2.393943,2.297430,3.833973,1.598859,1.401418,-3.648135,-0.587917,1.919953,4.711457,4.863253,2.481126,2.255025,1.979860,1.705557,1.559919,5.024892,4.370026,4.337346,3.083401,3.296542,5.083289,3.013304,-8.399359,2.802454,True
A005380_2016-12-08,0.676756,0.702852,0.702668,0.695379,0.710232,0.679352,0.772897,0.775099,0.780006,0.802263,0.800600,0.786126,0.765701,0.787155,0.764507,0.745534,0.717148,0.670724,0.656287,0.670834,0.651853,0.659183,0.629282,0.613264,0.721701,0.620271,0.608810,0.599695,0.590395,0.516193,0.463969,0.492637,0.502358,0.492291,0.453635,0.510191,0.454433,0.324001,0.287065,0.285971,...,-0.555524,3.837505,1.862922,1.271754,-5.785445,1.413928,-1.039940,-1.159445,-2.001940,-0.831339,1.689548,3.475641,1.752661,7.740514,-0.902877,-0.806831,-0.082521,-4.622880,1.316102,3.101742,-0.866607,1.850197,-1.975505,2.043032,-1.471736,2.729297,0.580684,2.952885,-2.354925,-0.263419,0.238883,1.764684,1.894234,0.063998,-1.942392,0.891554,1.722479,-3.422478,-1.238860,True
...,...,...,...,...,...,..

### 학습

In [21]:
train, test = train_test_split(BagOfBeta_df_230_droped, test_size=0.2, random_state=777)
train_x = train[train.columns[:-1]]
train_y = train[train.columns[-1]]
test_x = test[test.columns[:-1]]
test_y = test[test.columns[-1]]

In [22]:
rfc_model_230 = RandomForestClassifier(n_estimators=100, random_state=777, max_depth = 100, min_samples_split = 0.001)
rfc_model_230.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=100, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=0.001,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=777,
                       verbose=0, warm_start=False)

In [23]:
pred = rfc_model_230.predict(test_x)

print("Accuracy: %s" % str(rfc_model_230.score(test_x, test_y)))
print("Confusion Matrix")
print(confusion_matrix(pred, test_y, labels=[True, False])) 

Accuracy: 0.6433566433566433
Confusion Matrix
[[54 24]
 [27 38]]


## 230_460시점(2016-12-08, 2017-11-17) 학습 후 정확도 확인

In [50]:
period_list = [x*230 for x in range(1, 3)]
print(period_list)

[230, 460]


### 데이터셋 생성

In [51]:
%%time
#230, 460시점 생성
index_list = []
BoB_list = []
for period in period_list:
  print(period)
  for i, ticker in enumerate(pricedata_kospi.columns):
    print('\r%d/%d'%(i+1, len(pricedata_kospi.columns)), end='')
    state, getedBoB = getBoB(ticker, period)
    index_list.append(state)
    BoB_list.append(getedBoB)
  print()

230
784/784
460
784/784
CPU times: user 4min 10s, sys: 3.56 s, total: 4min 14s
Wall time: 4min 9s


In [52]:
BagOfBeta_df_230_460 = pd.DataFrame(BoB_list, index = index_list)

In [53]:
classified_list = []
for period in period_list:
  print(period)
  for i, ticker in enumerate(pricedata_kospi.columns):
    print('\r%d/%d'%(i+1, len(pricedata_kospi.columns)), end='')
    classified_list.append(getClassified(ticker, period))
  print()

230
784/784
460
784/784


In [54]:
BagOfBeta_df_230_460['result'] = classified_list
BagOfBeta_df_230_460_droped = BagOfBeta_df_230_460.dropna()

In [55]:
BagOfBeta_df_230_460_droped

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,result
A005930_2016-12-08,1.338226,1.338483,1.362913,1.357541,1.347650,1.390923,1.460837,1.462274,1.473762,1.451128,1.455293,1.461940,1.457865,1.424735,1.443555,1.462362,1.453106,1.490183,1.485212,1.488197,1.513174,1.532893,1.555864,1.600899,1.783054,1.767923,1.805102,1.804722,1.793388,1.796138,1.806284,1.764772,1.748480,1.754870,1.714398,1.427788,1.482753,1.451303,1.277072,1.300163,...,3.026271,0.749069,1.785682,0.927805,0.648878,2.555923,1.910395,2.558089,2.566363,2.253052,0.575459,1.648875,2.940480,3.602700,1.894304,3.192075,3.276317,2.110682,1.296920,0.638460,6.622136,1.504686,-0.070883,0.602847,1.239400,2.000374,1.681891,1.404396,3.992485,0.317403,1.186953,-0.770792,1.987424,4.161994,2.901080,3.051549,1.056148,4.917341,1.980648,True
A000660_2016-12-08,1.254750,1.242087,1.220905,1.143546,1.168918,1.143197,1.176132,1.193249,1.155881,1.148405,1.159449,1.140309,1.151419,1.177920,1.217678,1.230552,1.181741,1.136567,1.147498,1.154808,1.140827,1.130129,1.133423,1.093474,1.279581,1.252857,1.232169,1.243898,1.294308,1.291827,1.270836,1.254326,1.320058,1.321684,1.359132,1.158966,1.110178,1.219141,1.288386,1.343966,...,1.191958,0.079624,0.004111,2.813305,0.870361,3.978080,2.311774,3.508367,6.727803,4.494073,5.866851,4.207799,5.680777,1.967961,2.778888,1.444210,0.556872,2.596057,2.345116,0.681337,-3.748944,0.844256,2.630603,7.742632,-3.423024,-3.001194,3.330272,1.446278,8.090961,-1.354777,-6.291540,3.270467,1.801854,3.203848,1.669019,2.933238,0.257831,6.100980,1.263442,True
A035420_2016-12-08,0.861434,0.880686,0.942875,0.929793,1.022238,0.944639,0.941297,0.952954,0.872667,0.851025,0.862823,0.849210,0.825198,0.819878,0.834658,0.813038,0.837336,0.859475,0.865122,0.881773,0.882787,0.888311,0.861548,0.862927,0.990403,1.048864,1.090183,1.123572,1.138587,1.200190,1.136596,1.130692,1.124013,1.137770,1.104624,1.220170,1.203678,1.273073,1.140140,1.147087,...,2.878658,4.180823,0.768748,2.331106,-3.247676,-0.472207,2.249312,1.469208,0.507057,2.271955,1.748778,0.134949,1.943806,-3.400550,7.860462,2.068875,0.296454,3.092196,4.375335,1.639354,1.572983,3.632818,1.944998,8.086779,-3.652865,4.886665,2.332576,2.018358,0.104550,0.839243,3.246921,0.888813,-1.509459,1.284209,1.016063,-1.135750,2.174155,5.399600,-0.778535,False
A051910_2016-12-08,1.481199,1.455214,1.423273,1.493332,1.411062,1.378351,1.364039,1.351593,1.331118,1.348830,1.334074,1.355190,1.323500,1.293191,1.255131,1.274977,1.283548,1.271304,1.228539,1.232156,1.246313,1.239002,1.270530,1.253393,1.067992,0.989141,0.942065,0.932957,0.907033,0.827452,0.836157,0.856197,0.907921,0.934806,1.012658,1.163759,1.231378,1.381751,1.452017,1.403880,...,-0.853174,-0.804991,0.018259,3.268729,5.778220,-0.196287,2.634002,2.013340,4.998720,6.990801,4.079725,1.925127,2.555923,0.294914,1.933539,-2.393943,2.297430,3.833973,1.598859,1.401418,-3.648135,-0.587917,1.919953,4.711457,4.863253,2.481126,2.255025,1.979860,1.705557,1.559919,5.024892,4.370026,4.337346,3.083401,3.296542,5.083289,3.013304,-8.399359,2.802454,True
A005380_2016-12-08,0.676756,0.702852,0.702668,0.695379,0.710232,0.679352,0.772897,0.775099,0.780006,0.802263,0.800600,0.786126,0.765701,0.787155,0.764507,0.745534,0.717148,0.670724,0.656287,0.670834,0.651853,0.659183,0.629282,0.613264,0.721701,0.620271,0.608810,0.599695,0.590395,0.516193,0.463969,0.492637,0.502358,0.492291,0.453635,0.510191,0.454433,0.324001,0.287065,0.285971,...,-0.555524,3.837505,1.862922,1.271754,-5.785445,1.413928,-1.039940,-1.159445,-2.001940,-0.831339,1.689548,3.475641,1.752661,7.740514,-0.902877,-0.806831,-0.082521,-4.622880,1.316102,3.101742,-0.866607,1.850197,-1.975505,2.043032,-1.471736,2.729297,0.580684,2.952885,-2.354925,-0.263419,0.238883,1.764684,1.894234,0.063998,-1.942392,0.891554,1.722479,-3.422478,-1.238860,True
...,...,...,...,...,...,..

### 학습

In [56]:
train, test = train_test_split(BagOfBeta_df_230_460_droped, test_size=0.2, random_state=777)
train_x = train[train.columns[:-1]]
train_y = train[train.columns[-1]]
test_x = test[test.columns[:-1]]
test_y = test[test.columns[-1]]

In [57]:
%%time
rfc_model_230_460 = RandomForestClassifier(n_estimators=1000, random_state=777, max_depth = 500, min_samples_split = 0.001)
rfc_model_230_460.fit(train_x, train_y)

CPU times: user 7.45 s, sys: 31.9 ms, total: 7.49 s
Wall time: 7.49 s


In [58]:
pred = rfc_model_230_460.predict(test_x)

print("Accuracy: %s" % str(rfc_model_230_460.score(test_x, test_y)))
print("Confusion Matrix")
print(confusion_matrix(pred, test_y, labels=[True, False])) 

Accuracy: 0.8298611111111112
Confusion Matrix
[[127  31]
 [ 18 112]]


## 230_460_690_920시점 학습 후 1150시점으로 정확도 검증

In [59]:
period_list = [x*230 for x in range(1, 5)]
print(period_list)

[230, 460, 690, 920]


### 데이터셋 생성

In [60]:
%%time
#230, 460, 690, 920시점 생성
index_list = []
BoB_list = []
for period in period_list:
  print(period)
  for i, ticker in enumerate(pricedata_kospi.columns):
    print('\r%d/%d'%(i+1, len(pricedata_kospi.columns)), end='')
    state, getedBoB = getBoB(ticker, period)
    index_list.append(state)
    BoB_list.append(getedBoB)
  print()

230
784/784
460
784/784
690
784/784
920
784/784
CPU times: user 8min 23s, sys: 7.42 s, total: 8min 31s
Wall time: 8min 22s


In [61]:
BagOfBeta_df_230_460_690_920 = pd.DataFrame(BoB_list, index = index_list)

In [62]:
classified_list = []
for period in period_list:
  print(period)
  for i, ticker in enumerate(pricedata_kospi.columns):
    print('\r%d/%d'%(i+1, len(pricedata_kospi.columns)), end='')
    classified_list.append(getClassified(ticker, period))
  print()

230
784/784
460
784/784
690
784/784
920
784/784


In [63]:
BagOfBeta_df_230_460_690_920['result'] = classified_list
BagOfBeta_df_230_460_690_920_droped = BagOfBeta_df_230_460_690_920.dropna()

In [64]:
BagOfBeta_df_230_460_690_920_droped

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,result
A005930_2016-12-08,1.338226,1.338483,1.362913,1.357541,1.347650,1.390923,1.460837,1.462274,1.473762,1.451128,1.455293,1.461940,1.457865,1.424735,1.443555,1.462362,1.453106,1.490183,1.485212,1.488197,1.513174,1.532893,1.555864,1.600899,1.783054,1.767923,1.805102,1.804722,1.793388,1.796138,1.806284,1.764772,1.748480,1.754870,1.714398,1.427788,1.482753,1.451303,1.277072,1.300163,...,3.026271,0.749069,1.785682,0.927805,0.648878,2.555923,1.910395,2.558089,2.566363,2.253052,0.575459,1.648875,2.940480,3.602700,1.894304,3.192075,3.276317,2.110682,1.296920,0.638460,6.622136,1.504686,-0.070883,0.602847,1.239400,2.000374,1.681891,1.404396,3.992485,0.317403,1.186953,-0.770792,1.987424,4.161994,2.901080,3.051549,1.056148,4.917341,1.980648,True
A000660_2016-12-08,1.254750,1.242087,1.220905,1.143546,1.168918,1.143197,1.176132,1.193249,1.155881,1.148405,1.159449,1.140309,1.151419,1.177920,1.217678,1.230552,1.181741,1.136567,1.147498,1.154808,1.140827,1.130129,1.133423,1.093474,1.279581,1.252857,1.232169,1.243898,1.294308,1.291827,1.270836,1.254326,1.320058,1.321684,1.359132,1.158966,1.110178,1.219141,1.288386,1.343966,...,1.191958,0.079624,0.004111,2.813305,0.870361,3.978080,2.311774,3.508367,6.727803,4.494073,5.866851,4.207799,5.680777,1.967961,2.778888,1.444210,0.556872,2.596057,2.345116,0.681337,-3.748944,0.844256,2.630603,7.742632,-3.423024,-3.001194,3.330272,1.446278,8.090961,-1.354777,-6.291540,3.270467,1.801854,3.203848,1.669019,2.933238,0.257831,6.100980,1.263442,True
A035420_2016-12-08,0.861434,0.880686,0.942875,0.929793,1.022238,0.944639,0.941297,0.952954,0.872667,0.851025,0.862823,0.849210,0.825198,0.819878,0.834658,0.813038,0.837336,0.859475,0.865122,0.881773,0.882787,0.888311,0.861548,0.862927,0.990403,1.048864,1.090183,1.123572,1.138587,1.200190,1.136596,1.130692,1.124013,1.137770,1.104624,1.220170,1.203678,1.273073,1.140140,1.147087,...,2.878658,4.180823,0.768748,2.331106,-3.247676,-0.472207,2.249312,1.469208,0.507057,2.271955,1.748778,0.134949,1.943806,-3.400550,7.860462,2.068875,0.296454,3.092196,4.375335,1.639354,1.572983,3.632818,1.944998,8.086779,-3.652865,4.886665,2.332576,2.018358,0.104550,0.839243,3.246921,0.888813,-1.509459,1.284209,1.016063,-1.135750,2.174155,5.399600,-0.778535,False
A051910_2016-12-08,1.481199,1.455214,1.423273,1.493332,1.411062,1.378351,1.364039,1.351593,1.331118,1.348830,1.334074,1.355190,1.323500,1.293191,1.255131,1.274977,1.283548,1.271304,1.228539,1.232156,1.246313,1.239002,1.270530,1.253393,1.067992,0.989141,0.942065,0.932957,0.907033,0.827452,0.836157,0.856197,0.907921,0.934806,1.012658,1.163759,1.231378,1.381751,1.452017,1.403880,...,-0.853174,-0.804991,0.018259,3.268729,5.778220,-0.196287,2.634002,2.013340,4.998720,6.990801,4.079725,1.925127,2.555923,0.294914,1.933539,-2.393943,2.297430,3.833973,1.598859,1.401418,-3.648135,-0.587917,1.919953,4.711457,4.863253,2.481126,2.255025,1.979860,1.705557,1.559919,5.024892,4.370026,4.337346,3.083401,3.296542,5.083289,3.013304,-8.399359,2.802454,True
A005380_2016-12-08,0.676756,0.702852,0.702668,0.695379,0.710232,0.679352,0.772897,0.775099,0.780006,0.802263,0.800600,0.786126,0.765701,0.787155,0.764507,0.745534,0.717148,0.670724,0.656287,0.670834,0.651853,0.659183,0.629282,0.613264,0.721701,0.620271,0.608810,0.599695,0.590395,0.516193,0.463969,0.492637,0.502358,0.492291,0.453635,0.510191,0.454433,0.324001,0.287065,0.285971,...,-0.555524,3.837505,1.862922,1.271754,-5.785445,1.413928,-1.039940,-1.159445,-2.001940,-0.831339,1.689548,3.475641,1.752661,7.740514,-0.902877,-0.806831,-0.082521,-4.622880,1.316102,3.101742,-0.866607,1.850197,-1.975505,2.043032,-1.471736,2.729297,0.580684,2.952885,-2.354925,-0.263419,0.238883,1.764684,1.894234,0.063998,-1.942392,0.891554,1.722479,-3.422478,-1.238860,True
...,...,...,...,...,...,..

### 학습

In [65]:
train, test = train_test_split(BagOfBeta_df_230_460_690_920_droped, test_size=0.2, random_state=777)
train_x = train[train.columns[:-1]]
train_y = train[train.columns[-1]]
test_x = test[test.columns[:-1]]
test_y = test[test.columns[-1]]

In [66]:
%%time
rfc_model_230_460_690_920 = RandomForestClassifier(n_estimators=1000, random_state=777, max_depth = 500, min_samples_split = 0.001)
rfc_model_230_460_690_920.fit(train_x, train_y)

CPU times: user 15 s, sys: 52.3 ms, total: 15.1 s
Wall time: 15 s


In [67]:
pred = rfc_model_230_460_690_920.predict(test_x)

print("Accuracy: %s" % str(rfc_model_230_460_690_920.score(test_x, test_y)))
print("Confusion Matrix")
print(confusion_matrix(pred, test_y, labels=[True, False]))

Accuracy: 0.92573402417962
Confusion Matrix
[[304  32]
 [ 11 232]]


### 1150시점 으로 검증

#### 1150 데이터셋 생성

In [68]:
%%time
#1150시점 생성
period_list = [1150]
index_list = []
BoB_list = []
for period in period_list:
  print(period)
  for i, ticker in enumerate(pricedata_kospi.columns):
    print('\r%d/%d'%(i+1, len(pricedata_kospi.columns)), end='')
    state, getedBoB = getBoB(ticker, period)
    index_list.append(state)
    BoB_list.append(getedBoB)
  print()

1150
784/784
CPU times: user 2min 5s, sys: 1.96 s, total: 2min 7s
Wall time: 2min 5s


In [69]:
BagOfBeta_df_1150 = pd.DataFrame(BoB_list, index = index_list)

In [70]:
classified_list = []
for period in period_list:
  print(period)
  for i, ticker in enumerate(pricedata_kospi.columns):
    print('\r%d/%d'%(i+1, len(pricedata_kospi.columns)), end='')
    classified_list.append(getClassified(ticker, period))
  print()

1150
784/784


In [71]:
BagOfBeta_df_1150['result'] = classified_list
BagOfBeta_df_1150_droped = BagOfBeta_df_1150.dropna()

In [72]:
BagOfBeta_df_1150_droped

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,result
A005930_2020-09-08,1.012332,1.009569,1.007693,1.008128,1.005259,1.003147,1.003625,1.000287,0.997752,0.996466,0.987496,0.987297,0.986200,0.988972,0.986452,0.981181,0.974133,0.973962,0.970133,0.968262,0.966201,0.961464,0.975971,1.022672,1.095495,1.113819,1.124907,1.085770,1.105783,1.112079,1.105645,1.097424,1.125251,1.069996,1.048970,1.112904,1.108818,1.098787,1.083327,1.062313,...,3.026271,0.749069,1.785682,0.927805,0.648878,2.555923,1.910395,2.558089,2.566363,2.253052,0.575459,1.648875,2.940480,3.602700,1.894304,3.192075,3.276317,2.110682,1.296920,0.638460,6.622136,1.504686,-0.070883,0.602847,1.239400,2.000374,1.681891,1.404396,3.992485,0.317403,1.186953,-0.770792,1.987424,4.161994,2.901080,3.051549,1.056148,4.917341,1.980648,True
A000660_2020-09-08,1.125174,1.124807,1.118025,1.116653,1.114456,1.111974,1.113353,1.110742,1.108887,1.108554,1.091304,1.090955,1.092663,1.105692,1.103728,1.107527,1.102751,1.102988,1.101208,1.116927,1.099796,1.092879,1.148952,1.082659,1.041993,0.983161,0.987785,0.959819,0.969189,0.950454,0.965258,0.958652,0.960542,0.865868,0.841576,0.845341,0.857968,0.888773,0.925714,0.949317,...,1.191958,0.079624,0.004111,2.813305,0.870361,3.978080,2.311774,3.508367,6.727803,4.494073,5.866851,4.207799,5.680777,1.967961,2.778888,1.444210,0.556872,2.596057,2.345116,0.681337,-3.748944,0.844256,2.630603,7.742632,-3.423024,-3.001194,3.330272,1.446278,8.090961,-1.354777,-6.291540,3.270467,1.801854,3.203848,1.669019,2.933238,0.257831,6.100980,1.263442,True
A035420_2020-09-08,0.756143,0.758747,0.755372,0.753717,0.760564,0.761538,0.750172,0.749027,0.750417,0.750051,0.756330,0.754540,0.753049,0.746514,0.749013,0.741087,0.744277,0.757747,0.757511,0.753636,0.734800,0.707631,0.686014,0.597544,0.613912,0.584359,0.593572,0.545533,0.539003,0.532106,0.551603,0.586900,0.570747,0.720236,0.682051,0.687233,0.592659,0.548720,0.686839,0.535141,...,2.878658,4.180823,0.768748,2.331106,-3.247676,-0.472207,2.249312,1.469208,0.507057,2.271955,1.748778,0.134949,1.943806,-3.400550,7.860462,2.068875,0.296454,3.092196,4.375335,1.639354,1.572983,3.632818,1.944998,8.086779,-3.652865,4.886665,2.332576,2.018358,0.104550,0.839243,3.246921,0.888813,-1.509459,1.284209,1.016063,-1.135750,2.174155,5.399600,-0.778535,False
A051910_2020-09-08,1.363505,1.366519,1.368219,1.368938,1.372055,1.373331,1.376841,1.374373,1.376810,1.374166,1.389342,1.389369,1.391532,1.386146,1.386661,1.382437,1.380789,1.395093,1.402178,1.419564,1.412780,1.415880,1.422366,1.245068,1.285642,1.381168,1.387363,1.381051,1.393027,1.394743,1.359892,1.401019,1.370775,1.437475,1.492702,1.085100,1.011852,1.077658,1.045939,0.962195,...,-0.853174,-0.804991,0.018259,3.268729,5.778220,-0.196287,2.634002,2.013340,4.998720,6.990801,4.079725,1.925127,2.555923,0.294914,1.933539,-2.393943,2.297430,3.833973,1.598859,1.401418,-3.648135,-0.587917,1.919953,4.711457,4.863253,2.481126,2.255025,1.979860,1.705557,1.559919,5.024892,4.370026,4.337346,3.083401,3.296542,5.083289,3.013304,-8.399359,2.802454,True
A207940_2020-09-08,0.861212,0.862500,0.861083,0.865622,0.868066,0.870174,0.868045,0.867224,0.867176,0.870568,0.876616,0.876567,0.875904,0.863430,0.865576,0.896816,0.898291,0.890348,0.890821,0.868452,0.866107,0.898398,0.875681,0.617053,0.680918,0.609924,0.495518,0.466120,0.461347,0.358172,0.346240,0.361879,0.337759,0.360857,0.724663,0.804212,0.809852,0.844570,0.872559,0.820152,...,0.162678,6.453880,0.471566,-0.172309,4.186384,1.665601,0.292518,2.068120,-3.190769,0.639615,0.683197,1.521952,10.508638,1.398145,-0.424341,-2.367099,3.094234,-0.706140,2.536528,1.991557,5.271163,-0.113008,3.000080,5.806709,4.288867,1.971290,0.281435,0.082727,0.638322,4.078064,4.143288,-5.393259,3.069780,0.457867,4.934525,-1.292144,-5.279681,3.921147,8.577939,True
...,...,...,...,...,...,...,...,..

#### 위에서 학습한 각 모델들로 검증

In [73]:
test_x = BagOfBeta_df_1150_droped[BagOfBeta_df_1150_droped.columns[:-1]]
test_y = BagOfBeta_df_1150_droped[BagOfBeta_df_1150_droped.columns[-1]]

In [75]:
pred = rfc_model_230.predict(test_x)

print("Accuracy: %s" % str(rfc_model_230.score(test_x, test_y)))
print("Confusion Matrix")
print(confusion_matrix(pred, test_y, labels=[True, False]))

Accuracy: 0.7565337001375516
Confusion Matrix
[[279  84]
 [ 93 271]]


In [76]:
pred = rfc_model_230_460.predict(test_x)

print("Accuracy: %s" % str(rfc_model_230_460.score(test_x, test_y)))
print("Confusion Matrix")
print(confusion_matrix(pred, test_y, labels=[True, False])) 

Accuracy: 0.9284731774415406
Confusion Matrix
[[362  42]
 [ 10 313]]


In [77]:
pred = rfc_model_230_460_690_920.predict(test_x)

print("Accuracy: %s" % str(rfc_model_230_460_690_920.score(test_x, test_y)))
print("Confusion Matrix")
print(confusion_matrix(pred, test_y, labels=[True, False]))

Accuracy: 0.9614855570839065
Confusion Matrix
[[368  24]
 [  4 331]]


## 전체 학습(230, 460 690, 920, 1150)후 현 시점(2021-03-29) 데이터로 스크리닝

In [78]:
period_list = [x*230 for x in range(1, 6)]
print(period_list)

[230, 460, 690, 920, 1150]


### 데이터셋 생성

In [79]:
%%time
#230, 460, 690, 920, 1150시점 생성
index_list = []
BoB_list = []
for period in period_list:
  print(period)
  for i, ticker in enumerate(pricedata_kospi.columns):
    print('\r%d/%d'%(i+1, len(pricedata_kospi.columns)), end='')
    state, getedBoB = getBoB(ticker, period)
    index_list.append(state)
    BoB_list.append(getedBoB)
  print()

230
784/784
460
784/784
690
784/784
920
784/784
1150
784/784
CPU times: user 10min 28s, sys: 9.38 s, total: 10min 37s
Wall time: 10min 26s


In [80]:
BagOfBeta_df_230_460_690_920_1150 = pd.DataFrame(BoB_list, index = index_list)

In [81]:
classified_list = []
for period in period_list:
  print(period)
  for i, ticker in enumerate(pricedata_kospi.columns):
    print('\r%d/%d'%(i+1, len(pricedata_kospi.columns)), end='')
    classified_list.append(getClassified(ticker, period))
  print()

230
784/784
460
784/784
690
784/784
920
784/784
1150
784/784


In [82]:
BagOfBeta_df_230_460_690_920_1150['result'] = classified_list
BagOfBeta_df_230_460_690_920_1150_droped = BagOfBeta_df_230_460_690_920_1150.dropna()

In [83]:
BagOfBeta_df_230_460_690_920_1150_droped

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,result
A005930_2016-12-08,1.338226,1.338483,1.362913,1.357541,1.347650,1.390923,1.460837,1.462274,1.473762,1.451128,1.455293,1.461940,1.457865,1.424735,1.443555,1.462362,1.453106,1.490183,1.485212,1.488197,1.513174,1.532893,1.555864,1.600899,1.783054,1.767923,1.805102,1.804722,1.793388,1.796138,1.806284,1.764772,1.748480,1.754870,1.714398,1.427788,1.482753,1.451303,1.277072,1.300163,...,3.026271,0.749069,1.785682,0.927805,0.648878,2.555923,1.910395,2.558089,2.566363,2.253052,0.575459,1.648875,2.940480,3.602700,1.894304,3.192075,3.276317,2.110682,1.296920,0.638460,6.622136,1.504686,-0.070883,0.602847,1.239400,2.000374,1.681891,1.404396,3.992485,0.317403,1.186953,-0.770792,1.987424,4.161994,2.901080,3.051549,1.056148,4.917341,1.980648,True
A000660_2016-12-08,1.254750,1.242087,1.220905,1.143546,1.168918,1.143197,1.176132,1.193249,1.155881,1.148405,1.159449,1.140309,1.151419,1.177920,1.217678,1.230552,1.181741,1.136567,1.147498,1.154808,1.140827,1.130129,1.133423,1.093474,1.279581,1.252857,1.232169,1.243898,1.294308,1.291827,1.270836,1.254326,1.320058,1.321684,1.359132,1.158966,1.110178,1.219141,1.288386,1.343966,...,1.191958,0.079624,0.004111,2.813305,0.870361,3.978080,2.311774,3.508367,6.727803,4.494073,5.866851,4.207799,5.680777,1.967961,2.778888,1.444210,0.556872,2.596057,2.345116,0.681337,-3.748944,0.844256,2.630603,7.742632,-3.423024,-3.001194,3.330272,1.446278,8.090961,-1.354777,-6.291540,3.270467,1.801854,3.203848,1.669019,2.933238,0.257831,6.100980,1.263442,True
A035420_2016-12-08,0.861434,0.880686,0.942875,0.929793,1.022238,0.944639,0.941297,0.952954,0.872667,0.851025,0.862823,0.849210,0.825198,0.819878,0.834658,0.813038,0.837336,0.859475,0.865122,0.881773,0.882787,0.888311,0.861548,0.862927,0.990403,1.048864,1.090183,1.123572,1.138587,1.200190,1.136596,1.130692,1.124013,1.137770,1.104624,1.220170,1.203678,1.273073,1.140140,1.147087,...,2.878658,4.180823,0.768748,2.331106,-3.247676,-0.472207,2.249312,1.469208,0.507057,2.271955,1.748778,0.134949,1.943806,-3.400550,7.860462,2.068875,0.296454,3.092196,4.375335,1.639354,1.572983,3.632818,1.944998,8.086779,-3.652865,4.886665,2.332576,2.018358,0.104550,0.839243,3.246921,0.888813,-1.509459,1.284209,1.016063,-1.135750,2.174155,5.399600,-0.778535,False
A051910_2016-12-08,1.481199,1.455214,1.423273,1.493332,1.411062,1.378351,1.364039,1.351593,1.331118,1.348830,1.334074,1.355190,1.323500,1.293191,1.255131,1.274977,1.283548,1.271304,1.228539,1.232156,1.246313,1.239002,1.270530,1.253393,1.067992,0.989141,0.942065,0.932957,0.907033,0.827452,0.836157,0.856197,0.907921,0.934806,1.012658,1.163759,1.231378,1.381751,1.452017,1.403880,...,-0.853174,-0.804991,0.018259,3.268729,5.778220,-0.196287,2.634002,2.013340,4.998720,6.990801,4.079725,1.925127,2.555923,0.294914,1.933539,-2.393943,2.297430,3.833973,1.598859,1.401418,-3.648135,-0.587917,1.919953,4.711457,4.863253,2.481126,2.255025,1.979860,1.705557,1.559919,5.024892,4.370026,4.337346,3.083401,3.296542,5.083289,3.013304,-8.399359,2.802454,True
A005380_2016-12-08,0.676756,0.702852,0.702668,0.695379,0.710232,0.679352,0.772897,0.775099,0.780006,0.802263,0.800600,0.786126,0.765701,0.787155,0.764507,0.745534,0.717148,0.670724,0.656287,0.670834,0.651853,0.659183,0.629282,0.613264,0.721701,0.620271,0.608810,0.599695,0.590395,0.516193,0.463969,0.492637,0.502358,0.492291,0.453635,0.510191,0.454433,0.324001,0.287065,0.285971,...,-0.555524,3.837505,1.862922,1.271754,-5.785445,1.413928,-1.039940,-1.159445,-2.001940,-0.831339,1.689548,3.475641,1.752661,7.740514,-0.902877,-0.806831,-0.082521,-4.622880,1.316102,3.101742,-0.866607,1.850197,-1.975505,2.043032,-1.471736,2.729297,0.580684,2.952885,-2.354925,-0.263419,0.238883,1.764684,1.894234,0.063998,-1.942392,0.891554,1.722479,-3.422478,-1.238860,True
...,...,...,...,...,...,..

### 학습

In [88]:
train, test = train_test_split(BagOfBeta_df_230_460_690_920_1150_droped, test_size=0.2, random_state=777)

In [89]:
train_x = train[train.columns[:-1]]
train_y = train[train.columns[-1]]

In [90]:
test_x = test[test.columns[:-1]]
test_y = test[test.columns[-1]]

In [91]:
%%time
rfc_model_230_460_690_920_1150 = RandomForestClassifier(n_estimators=1000, random_state=777, max_depth = 500, min_samples_split = 0.001)
rfc_model_230_460_690_920_1150.fit(train_x, train_y)

CPU times: user 18.4 s, sys: 60.8 ms, total: 18.5 s
Wall time: 18.5 s


In [92]:
pred = rfc_model_230_460_690_920_1150.predict(test_x)

print("Accuracy: %s" % str(rfc_model_230_460_690_920_1150.score(test_x, test_y)))
print("Confusion Matrix")
print(confusion_matrix(pred, test_y, labels=[True, False]))

Accuracy: 0.9765193370165746
Confusion Matrix
[[363  10]
 [  7 344]]


### 스크리닝

In [ ]:
# 현 시점 데이터(2021-03-29)로 230_460_690_920_1150 모델로 스크리닝

#### 1284시점(21-03-29) 관측 베타 데이터 생성

In [93]:
%%time
index_list = []
BoB_list = []
#현시점(1284, 21-03-29) 생성
for i, ticker in enumerate(pricedata_kospi.columns):
  print('\r%d/%d'%(i+1, len(pricedata_kospi.columns)), end='')
  state, getedBoB = getBoB(ticker, 1284)
  index_list.append(state)
  BoB_list.append(getedBoB)
print()

784/784
CPU times: user 2min 6s, sys: 1.75 s, total: 2min 8s
Wall time: 2min 6s


In [94]:
BagOfBeta_df_1284 = pd.DataFrame(BoB_list, index = index_list)
BagOfBeta_df_1284

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90
A005930_2021-03-29,1.106133,1.113892,1.118470,1.116020,1.109219,1.119422,1.104290,1.091621,1.125060,1.117676,1.124064,1.114634,1.113777,1.112500,1.084575,1.095748,1.105371,1.149888,1.149955,1.140719,1.131481,1.129372,1.169481,1.175248,1.180178,1.177857,1.189324,1.189657,1.189241,1.160989,1.130839,1.116166,1.102417,1.107158,1.073561,1.055087,0.981675,0.964339,0.905363,0.985502,...,-2.057266,3.026271,0.749069,1.785682,0.927805,0.648878,2.555923,1.910395,2.558089,2.566363,2.253052,0.575459,1.648875,2.940480,3.602700,1.894304,3.192075,3.276317,2.110682,1.296920,0.638460,6.622136,1.504686,-0.070883,0.602847,1.239400,2.000374,1.681891,1.404396,3.992485,0.317403,1.186953,-0.770792,1.987424,4.161994,2.901080,3.051549,1.056148,4.917341,1.980648
A000660_2021-03-29,1.185804,1.190289,1.188326,1.194224,1.195779,1.207697,1.182111,1.168877,1.234005,1.242587,1.261591,1.271122,1.288220,1.292039,1.301755,1.330924,1.353400,1.367457,1.369407,1.376272,1.377073,1.400946,1.471830,1.490550,1.492794,1.502165,1.526663,1.552983,1.584743,1.521741,1.519305,1.458003,1.428449,1.440841,1.431156,1.380763,1.465670,1.738610,1.712376,2.063215,...,-0.039130,1.191958,0.079624,0.004111,2.813305,0.870361,3.978080,2.311774,3.508367,6.727803,4.494073,5.866851,4.207799,5.680777,1.967961,2.778888,1.444210,0.556872,2.596057,2.345116,0.681337,-3.748944,0.844256,2.630603,7.742632,-3.423024,-3.001194,3.330272,1.446278,8.090961,-1.354777,-6.291540,3.270467,1.801854,3.203848,1.669019,2.933238,0.257831,6.100980,1.263442
A035420_2021-03-29,0.760223,0.756119,0.760356,0.771768,0.797886,0.798580,0.842017,0.854122,0.835323,0.852552,0.844257,0.863424,0.857146,0.859407,0.871051,0.879616,0.902137,0.927575,0.927544,0.919345,0.927998,0.907358,0.926834,0.936394,0.954735,0.946132,0.934009,0.996234,1.027213,1.058971,1.071518,1.068131,1.087930,1.081093,1.140899,1.212581,1.156194,1.163384,1.206508,1.414105,...,-1.888271,2.878658,4.180823,0.768748,2.331106,-3.247676,-0.472207,2.249312,1.469208,0.507057,2.271955,1.748778,0.134949,1.943806,-3.400550,7.860462,2.068875,0.296454,3.092196,4.375335,1.639354,1.572983,3.632818,1.944998,8.086779,-3.652865,4.886665,2.332576,2.018358,0.104550,0.839243,3.246921,0.888813,-1.509459,1.284209,1.016063,-1.135750,2.174155,5.399600,-0.778535
A051910_2021-03-29,1.338697,1.347034,1.347610,1.332881,1.349076,1.325326,1.346856,1.381244,1.251113,1.246206,1.249205,1.263661,1.246870,1.232867,1.265727,1.253789,1.249632,1.297271,1.309443,1.302804,1.308988,1.239926,1.314900,1.308471,1.324320,1.347521,1.316015,1.281986,1.298870,1.372756,1.336627,1.351379,1.381760,1.379253,1.390381,1.342255,1.233773,1.255147,1.308801,1.451895,...,10.409339,-0.853174,-0.804991,0.018259,3.268729,5.778220,-0.196287,2.634002,2.013340,4.998720,6.990801,4.079725,1.925127,2.555923,0.294914,1.933539,-2.393943,2.297430,3.833973,1.598859,1.401418,-3.648135,-0.587917,1.919953,4.711457,4.863253,2.481126,2.255025,1.979860,1.705557,1.559919,5.024892,4.370026,4.337346,3.083401,3.296542,5.083289,3.013304,-8.399359,2.802454
A207940_2021-03-29,0.689753,0.689167,0.656946,0.653229,0.669976,0.659676,0.674299,0.752792,0.847327,0.859801,0.839987,0.856410,0.854617,0.846774,0.853037,0.840399,0.815127,0.857320,0.858452,0.864693,0.865071,0.849272,0.823249,0.827976,0.813432,0.796007,0.860330,0.796212,0.809643,0.827826,0.846713,0.845106,0.806592,0.801165,0.812392,0.843771,0.886633,0.944865,1.194151,1.173754,...,8.363244,0.162678,6.453880,0.471566,-0.172309,4.186384,1.665601,0.292518,2.068120,-3.190769,0.639615,0.683197,1.521952,10.508638,1.398145,-0.424341,-2.367099,3.094234,-0.706140,2.536528,1.991557,5.271163,-0.113008,3.000080,5.806709,4.288867,1.971290,0.281435,0.082727,0.638322,4.078064,4.143288,-5.393259,3.069780,0.457867,4.934525,-1.292144,-5.279681,3.921147,8.577939
...,...,...,...

In [95]:
BagOfBeta_df_1284_droped = BagOfBeta_df_1284.dropna()

In [96]:
BagOfBeta_df_1284_droped

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90
A005930_2021-03-29,1.106133,1.113892,1.118470,1.116020,1.109219,1.119422,1.104290,1.091621,1.125060,1.117676,1.124064,1.114634,1.113777,1.112500,1.084575,1.095748,1.105371,1.149888,1.149955,1.140719,1.131481,1.129372,1.169481,1.175248,1.180178,1.177857,1.189324,1.189657,1.189241,1.160989,1.130839,1.116166,1.102417,1.107158,1.073561,1.055087,0.981675,0.964339,0.905363,0.985502,...,-2.057266,3.026271,0.749069,1.785682,0.927805,0.648878,2.555923,1.910395,2.558089,2.566363,2.253052,0.575459,1.648875,2.940480,3.602700,1.894304,3.192075,3.276317,2.110682,1.296920,0.638460,6.622136,1.504686,-0.070883,0.602847,1.239400,2.000374,1.681891,1.404396,3.992485,0.317403,1.186953,-0.770792,1.987424,4.161994,2.901080,3.051549,1.056148,4.917341,1.980648
A000660_2021-03-29,1.185804,1.190289,1.188326,1.194224,1.195779,1.207697,1.182111,1.168877,1.234005,1.242587,1.261591,1.271122,1.288220,1.292039,1.301755,1.330924,1.353400,1.367457,1.369407,1.376272,1.377073,1.400946,1.471830,1.490550,1.492794,1.502165,1.526663,1.552983,1.584743,1.521741,1.519305,1.458003,1.428449,1.440841,1.431156,1.380763,1.465670,1.738610,1.712376,2.063215,...,-0.039130,1.191958,0.079624,0.004111,2.813305,0.870361,3.978080,2.311774,3.508367,6.727803,4.494073,5.866851,4.207799,5.680777,1.967961,2.778888,1.444210,0.556872,2.596057,2.345116,0.681337,-3.748944,0.844256,2.630603,7.742632,-3.423024,-3.001194,3.330272,1.446278,8.090961,-1.354777,-6.291540,3.270467,1.801854,3.203848,1.669019,2.933238,0.257831,6.100980,1.263442
A035420_2021-03-29,0.760223,0.756119,0.760356,0.771768,0.797886,0.798580,0.842017,0.854122,0.835323,0.852552,0.844257,0.863424,0.857146,0.859407,0.871051,0.879616,0.902137,0.927575,0.927544,0.919345,0.927998,0.907358,0.926834,0.936394,0.954735,0.946132,0.934009,0.996234,1.027213,1.058971,1.071518,1.068131,1.087930,1.081093,1.140899,1.212581,1.156194,1.163384,1.206508,1.414105,...,-1.888271,2.878658,4.180823,0.768748,2.331106,-3.247676,-0.472207,2.249312,1.469208,0.507057,2.271955,1.748778,0.134949,1.943806,-3.400550,7.860462,2.068875,0.296454,3.092196,4.375335,1.639354,1.572983,3.632818,1.944998,8.086779,-3.652865,4.886665,2.332576,2.018358,0.104550,0.839243,3.246921,0.888813,-1.509459,1.284209,1.016063,-1.135750,2.174155,5.399600,-0.778535
A051910_2021-03-29,1.338697,1.347034,1.347610,1.332881,1.349076,1.325326,1.346856,1.381244,1.251113,1.246206,1.249205,1.263661,1.246870,1.232867,1.265727,1.253789,1.249632,1.297271,1.309443,1.302804,1.308988,1.239926,1.314900,1.308471,1.324320,1.347521,1.316015,1.281986,1.298870,1.372756,1.336627,1.351379,1.381760,1.379253,1.390381,1.342255,1.233773,1.255147,1.308801,1.451895,...,10.409339,-0.853174,-0.804991,0.018259,3.268729,5.778220,-0.196287,2.634002,2.013340,4.998720,6.990801,4.079725,1.925127,2.555923,0.294914,1.933539,-2.393943,2.297430,3.833973,1.598859,1.401418,-3.648135,-0.587917,1.919953,4.711457,4.863253,2.481126,2.255025,1.979860,1.705557,1.559919,5.024892,4.370026,4.337346,3.083401,3.296542,5.083289,3.013304,-8.399359,2.802454
A207940_2021-03-29,0.689753,0.689167,0.656946,0.653229,0.669976,0.659676,0.674299,0.752792,0.847327,0.859801,0.839987,0.856410,0.854617,0.846774,0.853037,0.840399,0.815127,0.857320,0.858452,0.864693,0.865071,0.849272,0.823249,0.827976,0.813432,0.796007,0.860330,0.796212,0.809643,0.827826,0.846713,0.845106,0.806592,0.801165,0.812392,0.843771,0.886633,0.944865,1.194151,1.173754,...,8.363244,0.162678,6.453880,0.471566,-0.172309,4.186384,1.665601,0.292518,2.068120,-3.190769,0.639615,0.683197,1.521952,10.508638,1.398145,-0.424341,-2.367099,3.094234,-0.706140,2.536528,1.991557,5.271163,-0.113008,3.000080,5.806709,4.288867,1.971290,0.281435,0.082727,0.638322,4.078064,4.143288,-5.393259,3.069780,0.457867,4.934525,-1.292144,-5.279681,3.921147,8.577939
...,...,...,...

#### 230_460_690_920_1150 모델로 예측

In [97]:
pred = rfc_model_230_460_690_920_1150.predict(BagOfBeta_df_1284_droped)

In [98]:
BagOfBeta_df_1284_droped['pred'] = pred.tolist()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
BagOfBeta_df_1284_droped

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,pred
A005930_2021-03-29,1.106133,1.113892,1.118470,1.116020,1.109219,1.119422,1.104290,1.091621,1.125060,1.117676,1.124064,1.114634,1.113777,1.112500,1.084575,1.095748,1.105371,1.149888,1.149955,1.140719,1.131481,1.129372,1.169481,1.175248,1.180178,1.177857,1.189324,1.189657,1.189241,1.160989,1.130839,1.116166,1.102417,1.107158,1.073561,1.055087,0.981675,0.964339,0.905363,0.985502,...,3.026271,0.749069,1.785682,0.927805,0.648878,2.555923,1.910395,2.558089,2.566363,2.253052,0.575459,1.648875,2.940480,3.602700,1.894304,3.192075,3.276317,2.110682,1.296920,0.638460,6.622136,1.504686,-0.070883,0.602847,1.239400,2.000374,1.681891,1.404396,3.992485,0.317403,1.186953,-0.770792,1.987424,4.161994,2.901080,3.051549,1.056148,4.917341,1.980648,True
A000660_2021-03-29,1.185804,1.190289,1.188326,1.194224,1.195779,1.207697,1.182111,1.168877,1.234005,1.242587,1.261591,1.271122,1.288220,1.292039,1.301755,1.330924,1.353400,1.367457,1.369407,1.376272,1.377073,1.400946,1.471830,1.490550,1.492794,1.502165,1.526663,1.552983,1.584743,1.521741,1.519305,1.458003,1.428449,1.440841,1.431156,1.380763,1.465670,1.738610,1.712376,2.063215,...,1.191958,0.079624,0.004111,2.813305,0.870361,3.978080,2.311774,3.508367,6.727803,4.494073,5.866851,4.207799,5.680777,1.967961,2.778888,1.444210,0.556872,2.596057,2.345116,0.681337,-3.748944,0.844256,2.630603,7.742632,-3.423024,-3.001194,3.330272,1.446278,8.090961,-1.354777,-6.291540,3.270467,1.801854,3.203848,1.669019,2.933238,0.257831,6.100980,1.263442,True
A035420_2021-03-29,0.760223,0.756119,0.760356,0.771768,0.797886,0.798580,0.842017,0.854122,0.835323,0.852552,0.844257,0.863424,0.857146,0.859407,0.871051,0.879616,0.902137,0.927575,0.927544,0.919345,0.927998,0.907358,0.926834,0.936394,0.954735,0.946132,0.934009,0.996234,1.027213,1.058971,1.071518,1.068131,1.087930,1.081093,1.140899,1.212581,1.156194,1.163384,1.206508,1.414105,...,2.878658,4.180823,0.768748,2.331106,-3.247676,-0.472207,2.249312,1.469208,0.507057,2.271955,1.748778,0.134949,1.943806,-3.400550,7.860462,2.068875,0.296454,3.092196,4.375335,1.639354,1.572983,3.632818,1.944998,8.086779,-3.652865,4.886665,2.332576,2.018358,0.104550,0.839243,3.246921,0.888813,-1.509459,1.284209,1.016063,-1.135750,2.174155,5.399600,-0.778535,False
A051910_2021-03-29,1.338697,1.347034,1.347610,1.332881,1.349076,1.325326,1.346856,1.381244,1.251113,1.246206,1.249205,1.263661,1.246870,1.232867,1.265727,1.253789,1.249632,1.297271,1.309443,1.302804,1.308988,1.239926,1.314900,1.308471,1.324320,1.347521,1.316015,1.281986,1.298870,1.372756,1.336627,1.351379,1.381760,1.379253,1.390381,1.342255,1.233773,1.255147,1.308801,1.451895,...,-0.853174,-0.804991,0.018259,3.268729,5.778220,-0.196287,2.634002,2.013340,4.998720,6.990801,4.079725,1.925127,2.555923,0.294914,1.933539,-2.393943,2.297430,3.833973,1.598859,1.401418,-3.648135,-0.587917,1.919953,4.711457,4.863253,2.481126,2.255025,1.979860,1.705557,1.559919,5.024892,4.370026,4.337346,3.083401,3.296542,5.083289,3.013304,-8.399359,2.802454,True
A207940_2021-03-29,0.689753,0.689167,0.656946,0.653229,0.669976,0.659676,0.674299,0.752792,0.847327,0.859801,0.839987,0.856410,0.854617,0.846774,0.853037,0.840399,0.815127,0.857320,0.858452,0.864693,0.865071,0.849272,0.823249,0.827976,0.813432,0.796007,0.860330,0.796212,0.809643,0.827826,0.846713,0.845106,0.806592,0.801165,0.812392,0.843771,0.886633,0.944865,1.194151,1.173754,...,0.162678,6.453880,0.471566,-0.172309,4.186384,1.665601,0.292518,2.068120,-3.190769,0.639615,0.683197,1.521952,10.508638,1.398145,-0.424341,-2.367099,3.094234,-0.706140,2.536528,1.991557,5.271163,-0.113008,3.000080,5.806709,4.288867,1.971290,0.281435,0.082727,0.638322,4.078064,4.143288,-5.393259,3.069780,0.457867,4.934525,-1.292144,-5.279681,3.921147,8.577939,True
...,...,...,...,...,...,...,...,...,

In [100]:
screened_raw = BagOfBeta_df_1284_droped[BagOfBeta_df_1284_droped['pred']==True].index
screened_raw

Index(['A005930_2021-03-29', 'A000660_2021-03-29', 'A051910_2021-03-29',
       'A207940_2021-03-29', 'A005380_2021-03-29', 'A006400_2021-03-29',
       'A035720_2021-03-29', 'A068270_2021-03-29', 'A066570_2021-03-29',
       'A105560_2021-03-29',
       ...
       'A010420_2021-03-29', 'A011230_2021-03-29', 'A084870_2021-03-29',
       'A001210_2021-03-29', 'A001420_2021-03-29', 'A003280_2021-03-29',
       'A001770_2021-03-29', 'A002420_2021-03-29', 'A004920_2021-03-29',
       'A010580_2021-03-29'],
      dtype='object', length=369)

In [101]:
screened = []
for raw_name in screened_raw:
  screened.append(raw_name[:7])

In [102]:
print(screened)

['A005930', 'A000660', 'A051910', 'A207940', 'A005380', 'A006400', 'A035720', 'A068270', 'A066570', 'A105560', 'A036570', 'A003550', 'A009150', 'A086790', 'A003670', 'A011200', 'A011170', 'A009830', 'A004020', 'A006800', 'A161390', 'A032640', 'A000720', 'A139480', 'A019170', 'A011070', 'A071050', 'A034020', 'A011790', 'A010140', 'A000100', 'A006280', 'A008930', 'A128940', 'A006360', 'A028670', 'A023530', 'A004990', 'A016360', 'A039490', 'A005940', 'A112610', 'A204320', 'A020150', 'A012510', 'A007070', 'A008560', 'A081660', 'A047040', 'A028050', 'A009240', 'A010060', 'A030000', 'A000990', 'A064350', 'A006650', 'A006260', 'A007310', 'A007700', 'A003090', 'A010120', 'A111770', 'A003000', 'A005250', 'A185750', 'A069620', 'A031430', 'A004000', 'A181710', 'A139130', 'A032350', 'A161890', 'A013890', 'A192820', 'A192080', 'A023590', 'A214320', 'A009420', 'A042700', 'A001230', 'A002840', 'A003850', 'A005850', 'A241590', 'A005440', 'A079160', 'A071840', 'A057050', 'A006040', 'A039130', 'A005690'

# 스크린된 종목

In [104]:
len(screened)

369

In [103]:
print(kospi_list[screened].iloc[0].to_list())

['삼성전자', 'SK하이닉스', 'LG화학', '삼성바이오로직스', '현대차', '삼성SDI', '카카오', '셀트리온', 'LG전자', 'KB금융', '엔씨소프트', 'LG', '삼성전기', '하나금융지주', '포스코케미칼', 'HMM', '롯데케미칼', '한화솔루션', '현대제철', '미래에셋대우', '한국타이어앤테크놀로지', 'LG유플러스', '현대건설', '이마트', '신풍제약', 'LG이노텍', '한국금융지주', '두산중공업', 'SKC', '삼성중공업', '유한양행', '녹십자', '한미사이언스', '한미약품', 'GS건설', '팬오션', '롯데쇼핑', '롯데지주', '삼성증권', '키움증권', 'NH투자증권', '씨에스윈드', '만도', '일진머티리얼즈', '더존비즈온', 'GS리테일', '메리츠증권', '휠라홀딩스', '대우건설', '삼성엔지니어링', '한샘', 'OCI', '제일기획', 'DB하이텍', '현대로템', '대한유화', 'LS', '오뚜기', 'F&F', '대웅', 'LS ELECTRIC', '영원무역', '부광약품', '녹십자홀딩스', '종근당', '대웅제약', '신세계인터내셔날', '롯데정밀화학', 'NHN', 'DGB금융지주', '롯데관광개발', '한국콜마', '지누스', '코스맥스', '더블유게임즈', '다우기술', '이노션', '한올바이오파마', '한미반도체', '동국제강', '미원상사', '보령제약', '에스엘', '화승엔터프라이즈', '현대그린푸드', 'CJ CGV', '롯데하이마트', '현대홈쇼핑', '동원산업', '하나투어', '파미셀', '제주항공', '대한해운', '쿠쿠홀딩스', '제이콘텐트리', '한세실업', '유나이티드제약', '한전기술', '대신증권', '락앤락', '미래에셋생명', '동원F&B', 'LG하우시스', '동아에스티', '삼화콘덴서', '삼양식품', '유안타증권', '일양약품', 'NICE', '풀무원', '한화투자증권', '신성이엔지', 'JW중외제약', 'SPC삼립', 'BGF', '잇츠한불